In [ ]:
import numpy as np
import matplotlib.pyplot as plt
####################################exercise 1################################
#Read csv file and store data in numpy format
spambase_X = np.loadtxt('./spambase_X.csv',delimiter=',')
spambase_y = np.loadtxt('./spambase_y.csv',delimiter=',')
X = spambase_X.T
y = spambase_y
n,d = X.shape
w = np.zeros(d)
#define perceptron function
def perceptron(X,y,w,b=0,max_pass=500):
  global n
  mistake = np.zeros(max_pass)
  for t in range(max_pass):
    for i in range(n):
      if (y[i]*(w.T.dot(X[i]) + b) <= 0):
        w = w + y[i]*X[i]
        b = b + y[i]
        mistake[t] = mistake[t] + 1

  return mistake
#generate plot for exercise 1
m = perceptron(X,y,w)
plt.plot(m)
plt.title('number of passes vs. number of mistakes')
plt.xlabel('number of passes')
plt.ylabel('number of mistakes')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
####################################exercise 2################################
#Read csv file and store data in numpy format
house_X_train = np.loadtxt('./housing_X_train.csv',delimiter=',')
house_y_train = np.loadtxt('./housing_y_train.csv',delimiter=',')
house_X_test = np.loadtxt('./housing_X_test.csv',delimiter=',')
house_y_test = np.loadtxt('./housing_y_test.csv',delimiter=',')
#tranform to numpy array format
house_X_train = np.mat(house_X_train).T
house_X_test = np.mat(house_X_test).T
house_y_train = np.mat(house_y_train).T
house_y_test = np.mat(house_y_test).T
def closed_form(X,y,lam):
  n,d = X.shape
  n_ones = np.ones(y.shape)
  d_ones = np.ones((d,1))
  X_2 = X.T.dot(X) #d*d
  X_y = X.T.dot(y) #d*1
  for i in range(X_2.shape[0]):
    X_2[i,i] += 2*n*lam
  b_vec = np.dot(X.T,n_ones) #d*1
  X_2 = np.append(X_2,b_vec,axis=1)
  a = X
  a = np.append(a,n_ones,axis=1)
  a = n_ones.T.dot(a)
  b = n_ones.T*y
  a = np.append(X_2,a,axis=0)
  b = np.append(X_y,b,axis=0)
  ans = np.linalg.solve(a,b)
  w = ans[0:-1,0]
  b = ans[-1,0]
  return w,b

In [3]:
def gradient(X,y,w,b,lam):
  n,d = X.shape
  n_ones = np.ones(y.shape)
  alpha = X.dot(w)+b*n_ones-y
  dw = (1/n)*np.dot(X.T,alpha) + 2*lam*w
  db = (1/n)*np.dot(n_ones.T,alpha)
  return dw,db
def error_func(X,y,w,b):
  n = X.shape[0]
  ones = np.ones((n,1))
  alpha = np.dot(X,w) + b*ones - y
  return np.dot(alpha.T,alpha)/(2*n)
def loss_func(X,y,w,b,lam):
  return error_func(X,y,w,b) + lam*np.dot(w.T,w)

def ridge_regression(X,y,X_test,y_test,max_pass,eta,tol,lam):
  w = np.zeros((X.shape[1],1))
  b = 0
  loss_train = []
  for t in range(max_pass):
    dw,db = gradient(X,y,w,b,lam)
    w_new = w - eta*dw
    b_new = b - eta*db[0,0]
    loss_train.append(loss_func(X,y,w_new,b_new,lam)[0,0])
    if np.linalg.norm((w_new-w),ord = 1) <= tol:
      w = w_new
      b = b_new
      break
    w,b = w_new,b_new
  error_train = error_func(X,y,w_new,b_new)[0,0]
  error_test = error_func(X_test,y_test,w_new,b_new)[0,0]
  return loss_train,error_train,error_test,w,b
def standarization(X_train,X_test):
  X = np.vstack((X_train,X_test))
  n,d = X.shape
  m = np.zeros(d)
  sd = np.zeros(d)
  X_sd = np.zeros((n,d))
  for i in range(n):
    for j in range(d):
      m[j] = np.mean(X[:,j])
      sd[j] = np.std(X[:,j])
      X_sd[i,j] = (X[i,j]-m[j])/sd[j]
  X_train_sd = X_sd[0:X_train.shape[0]]
  X_test_sd = X_sd[X_train.shape[0]:n]
  return X_train_sd,X_test_sd

In [ ]:
import time as t
n_test,d_test = house_X_test.shape
n,d = house_X_train.shape
l = [0,10]
print('when input data is not standarized')
print('for closed form solution')
for lam in l:
  print('when lambda equals to {0}'.format(lam))
  start = t.time()
  w,b = closed_form(house_X_train,house_y_train,lam)
  end = t.time()
  time_diff = end - start
  print('time used equals to {0}'.format(time_diff))
  print('the train loss is {0}'.format(loss_func(house_X_train,house_y_train,w,b,lam)))
  print('the train error is {0}'.format(error_func(house_X_train,house_y_train,w,b)))
  print('the test error is {0}'.format(error_func(house_X_test,house_y_test,w,b)))
print('for gradient descent solution')
for lam in l:
  print('when lambda equals to {0}'.format(lam))
  start = t.time()
  loss_train,error_train,error_test,w,b = ridge_regression(house_X_train,house_y_train,house_X_test,house_y_test,max_pass=500,eta=0.01,tol=0.000001,lam=lam)
  end = t.time()
  time_diff = end -start
  print('time used equals to {0}'.format(time_diff))
  print('the train error is {0}'.format(error_func(house_X_train,house_y_train,w,b)))
  print('the test error is {0}'.format(error_func(house_X_test,house_y_test,w,b)))
  plt.figure()
  plt.plot(loss_train)
  plt.xlabel('passes')
  plt.ylabel('train loss')
  plt.title('passes vs. train loss')
  plt.show()
house_X_train,house_X_test = standarization(house_X_train,house_X_test)
#house_X_test = standarization(house_X_test)
print('when input data is standarized')
print('for closed form solution')
for lam in l:
  print('when lambda equals to {0}'.format(lam))
  start = t.time()
  w,b = closed_form(house_X_train,house_y_train,lam)
  end = t.time()
  time_diff = end - start
  print('time used equals to {0}'.format(time_diff))
  print('the train loss is {0}'.format(loss_func(house_X_train,house_y_train,w,b,lam)))
  print('the train error is {0}'.format(error_func(house_X_train,house_y_train,w,b)))
  print('the test error is {0}'.format(error_func(house_X_test,house_y_test,w,b)))
print('for gradient descent solution')
for lam in l:
  print('when lambda equals to {0}'.format(lam))
  start = t.time()
  loss_train,error_train,error_test,w,b = ridge_regression(house_X_train,house_y_train,house_X_test,house_y_test,max_pass=500,eta=0.01,tol=0.000001,lam=lam)
  end = t.time()
  time_diff = end -start
  print('time used equals to {0}'.format(time_diff))
  print('the train error is {0}'.format(error_func(house_X_train,house_y_train,w,b)))
  print('the test error is {0}'.format(error_func(house_X_test,house_y_test,w,b)))
  plt.figure()
  plt.plot(loss_train)
  plt.xlabel('passes')
  plt.ylabel('train loss')
  plt.title('passes vs. train loss')
  plt.show()

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
##########################exercise 3######################################
X_train_A = np.loadtxt('./X_train_A.csv',delimiter=',')
X_train_B = np.loadtxt('./X_train_B.csv',delimiter=',')
X_train_C = np.loadtxt('./X_train_C.csv',delimiter=',')
Y_train_A = np.loadtxt('./Y_train_A.csv',delimiter=',')
Y_train_B = np.loadtxt('./Y_train_B.csv',delimiter=',')
Y_train_C = np.loadtxt('./Y_train_C.csv',delimiter=',')
X_test_A = np.loadtxt('./X_test_A.csv',delimiter=',')
X_test_B = np.loadtxt('./X_test_B.csv',delimiter=',')
X_test_C = np.loadtxt('./X_test_C.csv',delimiter=',')
Y_test_A = np.loadtxt('./Y_test_A.csv',delimiter=',')
Y_test_B = np.loadtxt('./Y_test_B.csv',delimiter=',')
Y_test_C = np.loadtxt('./Y_test_C.csv',delimiter=',')
X_train_A = np.mat(X_train_A)
X_train_B = np.mat(X_train_B)
X_train_C = np.mat(X_train_C)
Y_train_A = np.mat(Y_train_A).T
Y_train_B = np.mat(Y_train_B).T
Y_train_C = np.mat(Y_train_C).T
X_test_A = np.mat(X_test_A)
X_test_B = np.mat(X_test_B)
X_test_C = np.mat(X_test_C)
Y_test_A = np.mat(Y_test_A).T
Y_test_B = np.mat(Y_test_B).T
Y_test_C = np.mat(Y_test_C).T
LAM= [1e-3, 1e-2, 0.5, 1, 2, 5, 10, 15, 20, 30, 50]
K = 10
def kfold(X,Y,k,i):
  n,d = X.shape
  #seperate n rows data into k parts, each part has num rows
  num = n // k 
  train = list(range(0, i*num)) + list(range((i+1)*num, n))
  test = list(range(i * num, (i + 1) * num if (i + 1) * num < n else n))
  X_train = np.ndarray((len(train), d))
  X_test = np.ndarray((len(test), d))
  Y_train = np.ndarray((len(train), 1))
  Y_test = np.ndarray((len(test), 1))
  count = 0
  for j in train:
    X_train[count] = X[j]
    Y_train[count] = Y[j]
    count += 1
  count = 0
  for k in test: 
    X_test[count] = X[k]
    Y_test[count] = Y[k]
    count += 1
  return X_train, X_test, Y_train, Y_test
def linear_regression(X,Y):
  reg = linear_model.LinearRegression()
  model = reg.fit(X,np.array(Y.T)[0])
  return model,np.append(reg.coef_, reg.intercept_)

def ridge_regression(X,Y,LAM,K):
  error = []
  for l in LAM:
    error_lam = []
    for i in range(K):
      X_train, X_test, Y_train, Y_test = kfold(X, Y, K, i)
      reg_i = linear_model.Ridge(alpha=l)
      model = reg_i.fit(X_train, np.array(Y_train.T)[0])
      Y_hat = model.predict(X_test)
      error_i = mean_squared_error(np.array(Y_test.T)[0], np.array(Y_hat))
      error_lam.append(error_i)
    mean_error = np.mean(error_lam)
    error.append(mean_error)
  min_error = min(error)
  min_index = error.index(min_error)
  min_lam = LAM[min_index]
  reg = linear_model.Ridge(alpha=min_lam)
  model = reg.fit(X, np.array(Y.T)[0]) 
  return model, np.append(reg.coef_, reg.intercept_), min_lam

def Lasso_regression(X,Y,LAM,K):
  error = []
  for l in LAM:
    error_lam = []
    for i in range(K):
      X_train, X_test, Y_train, Y_test = kfold(X, Y, K, i)
      reg_i =linear_model.Lasso(alpha=l)
      model = reg_i.fit(X_train, np.array(Y_train.T)[0])
      Y_hat = model.predict(X_test)
      error_i = mean_squared_error(np.array(Y_test.T)[0], np.array(Y_hat))
      error_lam.append(error_i)
    mean_error = np.mean(error_lam)
    error.append(mean_error)
  min_error = min(error)
  min_index = error.index(min_error)
  min_lam = LAM[min_index]
  reg = linear_model.Lasso(alpha=min_lam)
  model = reg.fit(X, np.array(Y.T)[0])
  return model, np.append(reg.coef_, reg.intercept_), min_lam
def test_error(model, X, Y):
  Y_hat = model.predict(X)
  error = mean_squared_error(np.array(Y.T)[0], np.array(Y_hat))
  return error

In [ ]:
model_A_linear, w_A_linear= linear_regression(X_train_A, Y_train_A)
model_A_ridge, w_A_ridge, lam_A_ridge = ridge_regression(X_train_A, Y_train_A, LAM, K)
model_A_lasso, w_A_lasso, lam_A_lasso = Lasso_regression(X_train_A, Y_train_A, LAM, K)
model_B_linear, w_B_linear = linear_regression(X_train_B, Y_train_B)
model_B_ridge, w_B_ridge, lam_B_ridge = ridge_regression(X_train_B, Y_train_B, LAM, K)
model_B_lasso, w_B_lasso, lam_B_lasso = Lasso_regression(X_train_B, Y_train_B, LAM, K)
model_C_linear, w_C_linear = linear_regression(X_train_C, Y_train_C)
model_C_ridge, w_C_ridge, lam_C_ridge = ridge_regression(X_train_C, Y_train_C, LAM, K)
model_C_lasso, w_C_lasso, lam_C_lasso = Lasso_regression(X_train_C, Y_train_C, LAM, K)
Error_A_linear = test_error(model_A_linear, X_test_A, Y_test_A)
Error_A_ridge = test_error(model_A_ridge, X_test_A, Y_test_A)
Error_A_lasso = test_error(model_A_lasso, X_test_A, Y_test_A)
Error_B_linear = test_error(model_B_linear, X_test_B, Y_test_B)
Error_B_ridge = test_error(model_B_ridge, X_test_B, Y_test_B)
Error_B_lasso = test_error(model_B_lasso, X_test_B, Y_test_B)
Error_C_linear = test_error(model_C_linear, X_test_C, Y_test_C)
Error_C_ridge = test_error(model_C_ridge, X_test_C, Y_test_C)
Error_C_lasso = test_error(model_C_lasso, X_test_C, Y_test_C)
#ridge vs. lasso
print('test error for A using linear: {0}'.format(Error_A_linear))
print('Best lambada for A using ridge: {0}'.format(lam_A_ridge))
print('test error for A using ridge: {0}'.format(Error_A_ridge))
print('Best lambada for A using lasso: {0}'.format(lam_A_lasso))
print('test error for A using lasso: {0}'.format(Error_A_lasso))
print('\n')
print('test error for B using linear: {0}'.format(Error_B_linear))
print('Best lambada for B using ridge: {0}'.format(lam_B_ridge))
print('test error for B using ridge: {0}'.format(Error_B_ridge))
print('Best lambada for B using lasso: {0}'.format(lam_B_lasso))
print('test error for B using lasso: {0}'.format(Error_B_lasso))
print('\n')
print('test error for C using linear: {0}'.format(Error_C_linear))
print('Best lambada for C using ridge: {0}'.format(lam_C_ridge))
print('test error for C using ridge: {0}'.format(Error_C_ridge))
print('Best lambada for C using lasso: {0}'.format(lam_C_lasso))
print('test error for C using lasso: {0}'.format(Error_C_lasso))
print('\n')
#plot histogram graph
plt.hist(w_A_linear, alpha = 0.5, label='linear') 
plt.hist(w_A_ridge, alpha = 0.5, label='ridge') 
plt.hist(w_A_lasso, alpha = 0.5, label='lasso')
plt.legend()
plt.title('appear times of w solved from 3 method with data A')
plt.show()
plt.hist(w_B_linear, alpha = 0.5, label='linear') 
plt.hist(w_B_ridge, alpha = 0.5, label='ridge') 
plt.hist(w_B_lasso, alpha = 0.5, label='lasso')
plt.legend()
plt.title('appear times of w solved from 3 method with data B')
plt.show()
plt.hist(w_C_linear, alpha = 0.5, label='linear') 
plt.hist(w_C_ridge, alpha = 0.5, label='ridge') 
plt.hist(w_C_lasso, alpha = 0.5, label='lasso')
plt.legend()
plt.title('appear times of w solved from 3 method with data C')
plt.show()

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
##############################exercise 4##################################
#Read csv file and store data in numpy format
X_train_D = np.loadtxt('./X_train_D.csv',delimiter=',')
X_train_E = np.loadtxt('./X_train_E.csv',delimiter=',')
X_train_F = np.loadtxt('./X_train_F.csv',delimiter=',')
Y_train_D = np.loadtxt('./Y_train_D.csv',delimiter=',')
Y_train_E = np.loadtxt('./Y_train_E.csv',delimiter=',')
Y_train_F = np.loadtxt('./Y_train_F.csv',delimiter=',')
X_test_D = np.loadtxt('./X_test_D.csv',delimiter=',')
X_test_E = np.loadtxt('./X_test_E.csv',delimiter=',')
X_test_F = np.loadtxt('./X_test_F.csv',delimiter=',')
Y_test_D = np.loadtxt('./Y_test_D.csv',delimiter=',')
Y_test_E = np.loadtxt('./Y_test_E.csv',delimiter=',')
Y_test_F = np.loadtxt('./Y_test_F.csv',delimiter=',')
#tranform to numpy array format
X_train_D = np.mat(X_train_D).T
X_train_E = np.mat(X_train_E).T
X_train_F = np.mat(X_train_F)
Y_train_D = np.mat(Y_train_D).T
Y_train_E = np.mat(Y_train_E).T
Y_train_F = np.mat(Y_train_F).T
X_test_D = np.mat(X_test_D).T
X_test_E = np.mat(X_test_E).T
X_test_F = np.mat(X_test_F)
Y_test_D = np.mat(Y_test_D).T
Y_test_E = np.mat(Y_test_E).T
Y_test_F = np.mat(Y_test_F).T

In [8]:
def KNN(X_in,X_dataSet,Y_dataSet,k):
  size = X_dataSet.shape[0]
  y_out = []
  for n in range(X_in.shape[0]):
    X_test = X_in[n][:]
    diff = np.tile(X_test,(size,1)) - X_dataSet
    diff = np.power(diff,2)
    dist = diff.sum(axis=1)
    dist = np.power(dist,0.5)
    sortIndex = np.argsort(dist,axis=0)
    sum_y = 0
    for j in range(k):
      sum_y += Y_dataSet[sortIndex[j,0],0]
    y_pred = sum_y/k
    y_out.append(y_pred)
  y_out = np.array(y_out)
  return np.mat(y_out).T
#least square solution is just closed form solution with lambda equals to 0
def closed_form(X,y,X_in,lam=0):
  n,d = X.shape
  n_ones = np.ones(y.shape)
  d_ones = np.ones((d,1))
  X_2 = X.T.dot(X) #d*d
  X_y = X.T.dot(y) #d*1
  for i in range(X_2.shape[0]):
    X_2[i,i] += 2*n*lam
  b_vec = np.dot(X.T,n_ones) #d*1
  X_2 = np.append(X_2,b_vec,axis=1)
  a = X
  a = np.append(a,n_ones,axis=1)
  a = n_ones.T.dot(a)
  b = n_ones.T*y
  a = np.append(X_2,a,axis=0)
  b = np.append(X_y,b,axis=0)
  ans = np.linalg.solve(a,b)
  w = ans[0:-1,0]
  b = ans[-1,0]
  ones = np.ones((X_in.shape[0],1))
  y_pred = np.dot(X_in,w) + b*ones
  return y_pred
def error_func(Y_pred, Y):
  return np.power(np.linalg.norm(Y_pred - Y, ord=2), 2) / len(Y)
def quest2(X_train,Y_train,X_test,Y_test):
  x_max = np.max(X_train)
  x_min = np.min(X_train)
  x_in = list(range(int(x_min / 0.01), int(x_max / 0.01)))
  x_in = np.reshape(x_in, (len(x_in), 1))
  x_in = x_in * 0.01
  least_y_pred = closed_form(X_train,Y_train,x_in)
  knn_1_pred = KNN(x_in,X_train,Y_train,1)
  knn_9_pred = KNN(x_in,X_train,Y_train,9)
  plt.scatter(np.array(X_train).flatten(),np.array(Y_train).flatten())
  plt.plot(x_in,least_y_pred,label = 'linear')
  plt.plot(x_in,knn_1_pred,label = 'knn_1')
  plt.plot(x_in,knn_9_pred,label = 'knn_9')
  plt.xlabel('input x')
  plt.ylabel('predict y')
  plt.legend()
  plt.title('input x vs predit y coming from least square regression and KNN regression with k=1 and k=9')
  plt.show()
  k_choose = []
  error = []
  for i in range(9):
    knn_pred = KNN(X_test,X_train,Y_train,(i+1))
    ans = error_func(knn_pred,Y_test)
    k_choose.append(i+1)
    error.append(ans)
  least_error = error_func(closed_form(X_train,Y_train,X_test),Y_test)
  plt.plot(range(1,10),np.ones(9)*least_error,label = 'linear')
  plt.bar(k_choose,error,alpha=0.4,label = 'knn')
  plt.legend()
  plt.title('test error for KNN with k from 1 to 9 and also for least square regression')
  plt.xlabel('k')
  plt.ylabel('test error')
  plt.show()
def quest3(X_train,Y_train,X_test,Y_test):
  k_choose = []
  error = []
  for i in range(9):
    knn_pred = KNN(X_test,X_train,Y_train,(i+1))
    ans = error_func(knn_pred,Y_test)
    k_choose.append(i+1)
    error.append(ans)
  least_error = error_func(closed_form(X_train,Y_train,X_test),Y_test)
  plt.plot(range(1,10),np.ones(9)*least_error,label = 'linear')
  plt.bar(k_choose,error,alpha=0.4,label = 'knn')
  plt.legend()
  plt.title('test error for KNN with k from 1 to 9 and also for least square regression')
  plt.xlabel('k')
  plt.ylabel('test error')
  plt.show()

In [ ]:
print('for data D')
print('\n')
quest2(X_train_D,Y_train_D,X_test_D,Y_test_D)
print('for data E')
print('\n')
quest2(X_train_E,Y_train_E,X_test_E,Y_test_E)
print('for data F')
print('\n')
quest3(X_train_F,Y_train_F,X_test_F,Y_test_F)